# G3Py: Generalized Graphical Gaussian Processes in Python

## Imports and Sunspots Dataset

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import g3py as g3
import pymc3 as pm
import theano as th
import theano.tensor as tt

g3.style_normal()
g3.style_widget()

psamples = 0.1
data = 'subspots'


if data == 'subspots':
    x, y = g3.data_sunspots()
else:
    x, y = g3.data_heart()


obs_j, x_obs, y_obs, test_j, x_test, y_test = g3.random_obs(x, y, psamples, plot=True)

## Define Prior Distribution

In [ ]:
gp = g3.GaussianProcess(space=x, location=g3.Bias(), kernel=g3.SE(), noisy=True)
gp.describe('Sunspots', 'YEAR', 'SUNACTIVITY')
gp.plot(samples=5)
g3.print(gp.params, gp.logp(gp.params))

## Default Posterior Distribution, given observations (automatic parameters)

In [ ]:
gp.observed(inputs=x_obs, outputs=y_obs, hidden=y)
gp.plot(samples=5)
g3.print(gp.params, gp.logp(gp.params), gp.scores(gp.params))

## Manual Search of parameters

In [ ]:
gp.widget(samples=5)

## Change style and get widget params

In [ ]:
g3.style_big_seaborn()
gp.plot(gp.params_widget, samples=5)
#gp.eval_params(gp.params_widget)

## Default find_MAP with derivative methods and free-derivative methods

In [ ]:
params_map = gp.find_MAP(points=2)

In [ ]:
#g3.print(gp.eval_params(params_map))
gp.plot(params_map, samples=5, prior=True, title='Prior')
g3.show()
gp.plot(params_map, samples=5, title='Posterior')
g3.show()

## Find MAP from random points

In [ ]:
for k in range(5):
    init_params = gp.params_random(gp.params_widget, sigma=0.2)
    params = gp.find_MAP(init_params, points=2, display=False)
    #g3.print(gp.eval_params(params))
    gp.plot(params, samples=5)
    g3.show()

## Get prediction and custom plot

In [ ]:
prediction = gp.predict(params, samples=20)
mu, std, samples = prediction.mean, prediction.std, prediction.samples
g3.plot(samples, alpha=0.5)
g3.plot(mu, 'r', label='mean')
g3.plot(mu + 2*std, '--k', label='4 std')
g3.plot(mu - 2*std, '--k')
g3.plot_text('Example GP', 'Time', 'Measure')
g3.plot_save('images/gp_sunspots3.pdf')

# Sampling Hyperparameters with Ensemble MCMC

In [ ]:
datatrace = gp.sample_hypers(start=params, samples=10000, chains=10)
datatrace

## Convergence Diagnostics

In [ ]:
g3.style_seaborn()
g3.plot_datatrace(datatrace)

## Plot Marginals and reference parameters

In [ ]:
g3.hist_datatrace(datatrace, reference=gp.eval_params(params))

## Clustering of parameters and plot Bivariate distributions

In [ ]:
g3.cluster_datatrace(gp, datatrace)

In [ ]:
g3.scatter_datatrace(datatrace)

## Model Selection

In [ ]:
candidates = g3.find_candidates(datatrace, ll=1, by_cluster=True)
candidates

In [ ]:
gp.plot_datatrace(candidates)

In [ ]:
gp.plot_datatrace(candidates, overlap=True, limit=5, var=False, noise=False, samples=0, data=False, loc=True)

## Model Average

In [ ]:
average = gp.average(candidates, quantiles=True, quantiles_noise=True)
average

In [ ]:
gp.plot(values=average)

## Sampling Particles

In [ ]:
particles = gp.particles(candidates, nsamples=25)
g3.plot(particles, alpha=0.5)

In [ ]:
g3.plot(particles.mean(axis=1), label='Particles Mean')
gp.plot(values=average)

## Conditional and Marginal Datatrace as Empirical Distribution

In [ ]:
selected = g3.conditional_datatrace(datatrace, lambda df: df._ll > df._ll.quantile(0.9))
selected = g3.marginal_datatrace(selected, drop='_cluster')
g3.scatter_datatrace(selected)

## Kernel Density Estimation of Datatrace for sampling

In [ ]:
kde = g3.datatrace_to_kde(gp, selected)
models = g3.kde_to_datatrace(gp, kde, nsamples=25)
models = g3.marginal_datatrace(models, drop='_cluster')
g3.scatter_datatrace(models)

In [ ]:
average = gp.average(models, quantiles=True, quantiles_noise=True)
particles = gp.particles(models)

g3.plot(average.mean, label='Average Mean')
g3.plot(particles.mean(axis=1), label='Particles Mean')
gp.plot(params_map)
g3.plot(particles, alpha=0.1)